In [4]:
from riak import RiakClient
from datetime import datetime
import calendar
import csv
def changetime(stime):
            dt=datetime.strptime(stime,'%Y-%m-%dT%H:%M:%S')
            #print dt
            return calendar.timegm(datetime.timetuple(dt))*1000

/home/stephen/aarhus/local/lib/python2.7/site-packages/riak/security.py:45: UserWarning: OpenSSL 1.0.1f 6 Jan 2014 (>= 1.0.1g required), TLS 1.2 support: False
  warnings.warn(msg, UserWarning)


In [6]:
c=RiakClient()
c.ping()

True

In [7]:
#create table

fmt="""
    CREATE TABLE aarhus (
        family varchar not null,
        extid varchar not null,
        ts timestamp not null,
        avgMeasuredTime sint64 not null,
        avgSpeed sint64 not null,
        medianMeasuredTime sint64 not null,
        vehicleCount sint64 not null,
        id sint64 not null,
        report_id sint64 not null,
        PRIMARY KEY(
            (family,extid,quantum(ts,30,'d')),family,extid,ts)
            )
"""
c.ts_query('aarhus',fmt)

In [8]:
t=c.table('aarhus')
to=t.describe()

In [9]:
for r in to.rows:
    print r

['family', 'varchar', False, 1L, 1L]
['extid', 'varchar', False, 2L, 2L]
['ts', 'timestamp', False, 3L, 3L]
['avgMeasuredTime', 'sint64', False, None, None]
['avgSpeed', 'sint64', False, None, None]
['medianMeasuredTime', 'sint64', False, None, None]
['vehicleCount', 'sint64', False, None, None]
['id', 'sint64', False, None, None]
['report_id', 'sint64', False, None, None]


Need to change the n_val using the riak-admin cli

$riak-admin bucket-type update aarhus '{"props":{"n_val":1}}'
aarhus updated

$ riak-admin bucket-type status aarhus
aarhus is active

active: true
allow_mult: true
basic_quorum: false
big_vclock: 50
chash_keyfun: {riak_core_util,chash_std_keyfun}
claimant: 'riak@127.0.0.1'
ddl: {ddl_v1,<<"aarhus">>,
             [{riak_field_v1,<<"family">>,1,varchar,false},
              {riak_field_v1,<<"extid">>,2,varchar,false},
              {riak_field_v1,<<"ts">>,3,timestamp,false},
              {riak_field_v1,<<"avgMeasuredTime">>,4,sint64,false},
              {riak_field_v1,<<"avgSpeed">>,5,sint64,false},
              {riak_field_v1,<<"medianMeasuredTime">>,6,sint64,false},
              {riak_field_v1,<<"vehicleCount">>,7,sint64,false},
              {riak_field_v1,<<"id">>,8,sint64,false},
              {riak_field_v1,<<"report_id">>,9,sint64,false}],
             {key_v1,[{param_v1,[<<"family">>]},
                      {param_v1,[<<"extid">>]},
                      {hash_fn_v1,riak_ql_quanta,quantum,
                                  [{param_v1,[<<"ts">>]},30,d],
                                  timestamp}]},
             {key_v1,[{param_v1,[<<"family">>]},
                      {param_v1,[<<"extid">>]},
                      {param_v1,[<<"ts">>]}]}}
dvv_enabled: true
dw: quorum
last_write_wins: false
linkfun: {modfun,riak_kv_wm_link_walker,mapreduce_linkfun}
n_val: 1
notfound_ok: true
old_vclock: 86400
postcommit: []
pr: 0
precommit: []
pw: 0
r: quorum
rw: quorum
small_vclock: 50
w: quorum
write_once: true
young_vclock: 20


In [ ]:
#to load data in the table

count=0
ds=[]
t=c.table('aarhus')
print t


with open('./traffic_feb_june/all-data-2.csv', 'rU') as infile:
    r=csv.reader(infile)
    for l in r:
        newl=['1',str(l[3]),changetime(l[5]),int(l[1]),int(l[2]),int(l[4]),int(l[6]),int(l[7]),int(l[8])]
        count=count+1
        ds.append(newl)
infile.close()
to=t.new(ds)
print "Storage result:  ",to.store()
print count

aarhus
Storage result:  

In [ ]:
print c.query('aarhus','DESCRIBE aarhus')